In [ ]:
from util_plot import *
from util_feature import * 
from util_other import *
from util_model import * 

In [ ]:
file_path = 'data/shopping_time'

In [ ]:
# Unzip all the docs as csv obviously

neighbourhoods = file_path+'neighbourhoods.csv'
listings = file_path+'listings.csv'
listings_summary = file_path+'listings_summary.csv'
reviews = file_path+'reviews.csv'
reviews_summary = file_path+'reviews_summary.csv'

In [ ]:
df_neighb = pd.read_csv(neighbourhoods, delimiter=',')
df_list = pd.read_csv(listings, delimiter=',')
df_list_sum = pd.read_csv(listings_summary, delimiter=',')
df_rev = pd.read_csv(reviews, delimiter=',')
df_rev_sum = pd.read_csv(reviews_summary, delimiter=',')

list_dataframes = [df_neighb, df_list, df_list_sum, df_rev, df_rev_sum]

In [ ]:
####  USE THIS
colnum =
colcat =
coldate =
coltext =  []
colcat_1hot




Only looking at listings.csv, to get an idea of the functions

Now work on listings_summary.csv

As was seen in the plotting of the price, there are huge outliers in the price (some apartments are very expensive, but there are very few of those)

plot_col_distribution(df_list_sum_tronc, ['price'])

plot_cols_with_NaNs(df_list_sum_tronc, 10)

Following this, we can remove the columns with too many missing values

plot_col_correl_matrix(df_list_sum_tronc, df_list_sum_tronc.columns, annot=False)

columns_to_keep = ['id', 'host_has_profile_pic','neighbourhood_group_cleansed', 
                   'latitude', 'longitude','property_type','room_type','accommodates','bed_type', 'price', 'cleaning_fee', 
                   'security_deposit', 'extra_people', 'guests_included', 'minimum_nights',  
                   'instant_bookable', 'is_business_travel_ready', 'cancellation_policy']

plot_col_correl_target(df_list_sum_tronc, columns_to_keep, ['price'], nb_to_show=10, ascending=False)

plot_col_correl_target(df_list_sum_tronc, df_list_sum_tronc.columns, ['price'], nb_to_show=5, ascending=True)

__Feature Engineering__

In [ ]:
# Calculating the distance from the appartement to the center of Berlin

berlin_center = (52.5027778, 13.404166666666667)
df_list_sum_tronc = pd_col_add_distance_to_point(df_list_sum_tronc, berlin_center)

In [ ]:
pd_coltext_word_frequency(df_list_sum_tronc, ['amenities'], nb_to_show=30)

In [ ]:
# Add One hot encoding for some features in the appartment from what is inside 'amenities'

df_list_sum_tronc = pd_coltext_extract_tag(df_list_sum_tronc, 'Wifi', 'amenities')
df_list_sum_tronc = pd_coltext_extract_tag(df_list_sum_tronc, 'Laptop friendly workspace', 'amenities')
df_list_sum_tronc = pd_coltext_extract_tag(df_list_sum_tronc, 'TV', 'amenities')
df_list_sum_tronc = pd_coltext_extract_tag(df_list_sum_tronc, 'Free street parking', 'amenities')
df_list_sum_tronc = pd_coltext_extract_tag(df_list_sum_tronc, 'Elevator', 'amenities')
df_list_sum_tronc = pd_coltext_extract_tag(df_list_sum_tronc, 'Family/kid friendly', 'amenities')

In [ ]:
# After that, just drop 'amenities' col:
#df_list_sum_tronc = pd_col_remove(df_list_sum_tronc, ['amenities'])

In [ ]:
text = ['good', 'cozy', 'quiet', 'far', 'big', 'large', 'close', 'family', 'calm']

In [ ]:
df_list_sum_tronc = pd_coltext_tfidf(df_list_sum_tronc, 'space', text)

In [ ]:
col_keep = ['id', 'host_has_profile_pic', 'neighbourhood_group_cleansed', 
                   'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bed_type', 'price', 'cleaning_fee', 
                   'security_deposit', 'extra_people', 'guests_included', 'minimum_nights',  
                   'instant_bookable', 'is_business_travel_ready', 'cancellation_policy']

In [ ]:
col_keep = col_keep + text

In [ ]:
# Removing the 6 rows that have missing values to be able to do Logistic Regression
df_list_sum_tronc = df_list_sum_tronc[col_keep].dropna(axis=0)

In [ ]:
#df_list_sum_tronc[columns_to_keep].loc[df_list_sum_tronc[columns_to_keep].isnull().any(axis=1)]

In [ ]:
plot_cols_with_NaNs(df_list_sum_tronc, 5)

In [ ]:
# For modelling, removing outliers

df_list_sum_tronc = pd_row_drop_above_thresh(df_list_sum_tronc, 'price', thresh=500)

In [ ]:
df_list_sum_tronc = pd_df_one_hot_encode(df_list_sum_tronc,  df_list_sum_tronc.columns)

In [ ]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn.pipeline import Pipeline
#from sklearn.model_selection import GridSearchCV
#from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import mean_absolute_error, make_scorer

In [ ]:
def split_train_test(df, target, split_ratio=0.2):
    
    train_X, val_X, train_y, val_y = train_test_split(df,
                                                 df[target],
                                                 test_size = split_ratio,
                                                 random_state=42,
                                                 shuffle=False)
    print('train_X shape:',train_X.shape)
    print('val_X shape:',val_X.shape)

    print('train_y shape:',train_y.shape)
    print('val_y shape:',val_y.shape)
    
    return train_X, val_X, train_y, val_y

In [ ]:
train_X, val_X, train_y, val_y = split_train_test(df_list_sum_tronc, 'price')

In [ ]:
#pipeline = Pipeline([
#    ('std_scaler', StandardScaler())
#])

#train_X_scale = pipeline.fit_transform(train_X)
#val_X_scale = pipeline.fit(val_X)

In [ ]:
# I don't know if you wanted to add this, so I simply put it here, to avoid getting warning messages during training

import warnings
warnings.filterwarnings("ignore")

In [ ]:
logReg = LogisticRegression()
model = logReg.fit(train_X, train_y)

In [ ]:
clf = LogisticRegression(random_state=42)

model, train_pred, val_pred = sk_model_eval(clf, train_X, train_y, val_X, val_y)

